In [1]:
import glob
import networkx
import pandas as pd

#### Load Dataset

The Nest class in the *termite* module provides tools for preprocessing a tracking result dataset, we'll assume that the data was generated using this process from now on.

Let's proceed to load the data then...

In [5]:
def load_dataset(datapath):
    dataset = []
    for table in glob.glob(datapath):
        dataset.append(pd.read_csv(table))
    return dataset

dataset = load_dataset('../data/Clustering/expanded/*.csv')

This gives us a list of Pandas dataframes containg each termite data, from now on we can further investigate the underlying structure of termite self organization.